# [Advent of Code - 2017](https://adventofcode.com/2017)

Advent of code is a puzzle solving website, two puzzles released for each day of advent (Dec 1st - Dec 25th). If previous years are anything to go by the cover a large variety of algorithms and are generally quite fun!

Each year the puzzels are built around a central theme, with this year's theme being that we have been "digitized" into a computer, and must solve various problems from inside the machine.

# Day 0

This portion contains various common pieces of code that'll be used on multiple days.

In [70]:
from collections import Counter, defaultdict
from itertools import cycle, count
import os
import re


cat = ''.join
        
def Input(day):
    """Fetch the data input from disk."""
    filename = os.path.join('../data/advent2017/input{}.txt'.format(day))
    return open(filename)


def neighbours4(x, y):
    return (x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)


def neighbours8(x, y):
    return (
        (x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1),
        (x - 1, y - 1), (x - 1, y + 1), (x + 1, y - 1), (x + 1, y + 1)
    )


def manhattan_distance(point1, point2):
    return abs(point2[0] - point1[0]) + abs(point2[1] - point1[1])

# TREES

class Node(object):
    def __init__(self, name):
        self.name = name
        self._children = []
        self.parent = None
        
    def __repr__(self):
        return '<Node 0x:{} name={}>'.format(
            id(self),
            self.name
        )
    
    @property
    def children(self):
        """Immutable set of the node's children."""
        return tuple(self._children)
    
    def add_child(self, child):
        self._children.append(child)
    
    def remove_child(self, child):
        self._children.remove(child)
    
    def add_parent(self, parent):
        if self.parent:
            self.parent.remove_child(self)

        self.parent = parent

        if self.parent:
            self.parent.add_child(self)
    
    @property
    def root(self):
        if not self.parent:
            return self
        return self.parent.root
    
    @property
    def decendants(self):
        """Visit all children."""
        iterator = visit_pre_order(self)
        # Skip this node
        next(iterator)
        return iterator
    
    @property
    def siblings(self):
        for child in self.parent.children:
            if child == self:
                continue
            yield child

    
def visit_pre_order(node):
    """Visit a Tree in pre-order.
    
      A
     / \
    B   C
    
    A -> B -> C
    """
    yield node
    for child in node.children:
        yield from visit_pre_order(child)

# [Day 1: Inverse Captcha](https://adventofcode.com/2017/day/1)

We're greeted by a door, and must proove that we are _not_ human to continue. The first puzzle has us performing a captha that "only a computer" can solve. We're required to sum digits in a list where each digit matches the one immediately following it, wrapping to the start if we overflow.

In [2]:
def sum_if_match(nums, jump_distance):
    total = 0
    for index, n in enumerate(nums):
        next_n = nums[(index + jump_distance) % len(nums)] 
        if n == next_n:
            total += n
    return total

def sum_consecutive(data):
    nums = list(map(int, data))
    jump_distance = 1
    return sum_if_match(nums, jump_distance)


assert sum_consecutive('1122') == 3
assert sum_consecutive('1111') == 4
assert sum_consecutive('1234') == 0
assert sum_consecutive('91212129') == 9

sum_consecutive(Input(1).read().strip())

1182

For the second portion, we're again summing digits, but now only if we match the digit exactly _half the list_ away. At this point we can modify the initial code and provide a jump distance.

In [3]:
def sum_half(data):
    nums = list(map(int, data))
    jump_distance = len(nums) // 2
    return sum_if_match(nums, jump_distance)
    

assert sum_half('1212') == 6
assert sum_half('1221') == 0
assert sum_half('123425') == 4
assert sum_half('123123') == 12
assert sum_half('12131415') == 4

sum_half(Input(1).read().strip())

1152

# [Day 2: Corruption Checksum](https://adventofcode.com/2017/day/2)

Here we're required to perform some data anaylsis on a spreadsheet calculating a checksum of each row by finding the difference of the max and min values.

In [4]:
def parse_input(data):
    as_ints = []
    for row in data.split('\n'):
        if not row:
            break
        nums = list(map(int, re.findall('\d+', row)))
        as_ints.append(nums)
    return as_ints


data = parse_input(Input(2).read())

sum((max(row) - min(row) for row in data))

42378

The second portion requires us to find pairs of number in each row that are evenly divisible, and summing the result of their division. As we don't know in which order the pair will appear, I sort each row when meeting it.

In [5]:
def sum_even_div(data):
    total = 0
    for nums in data:
        nums = sorted(nums)
        for i, x in enumerate(nums[:-1]):
            for y in nums[i + 1:]:
                if y % x == 0:
                    total += y // x
                    break
    return total


sum_even_div(data)

246

# [Day 3: Spiral Memory](https://adventofcode.com/2017/day/3)

We need to find the coordinate of an number when it's displayed in a spiral format. I.e.

```
17  16  15  14  13
18   5   4   3  12
19   6   1   2  11
20   7   8   9  10
21  22  23---> ...
```

I started this problem by working out an equation to find the location of the Nth element, without building the rest of the grid. While this worked well for the first part of the problem, the second part requires us to build a spiral grid anyway! (It's much smaller, but still.)

In [105]:
def find_coorindates(N):
    """Find the coordinates of N in a spiral matrix.
    
    We can find the shell that the number occurs in by
    finding the upper limit for each shell and 
    """
    # First find the shell that the number appears in
    # number of elements in each shell is 4*(n-1)
    if N == 1:
        return 0, 0

    shell = 1
    shell_size = 1
    while N > shell_size ** 2:
        shell += 1
        shell_size += 2

    shell_end = shell_size ** 2
    shell_start = (shell_size - 2) ** 2
    elms_in_shell = shell_end - shell_start
    position_in_shell = N - shell_start
    
    side_length = elms_in_shell // 4
    half_side = side_length // 2
    
    side = position_in_shell / elms_in_shell
    
    if side <= 0.25:
        # right
        x = half_side
        y = (position_in_shell % side_length) - half_side
    elif side <= 0.5:
        # top
        x = (position_in_shell % side_length) - half_side
        y = half_side
    elif side <= 0.75:
        # left
        x = -half_side
        y = (position_in_shell % side_length) - half_side
    else:
        # bottom
        x = (position_in_shell % side_length) - half_side
        y = -half_side
    return (x, y)
    
    
def carry_distance(N):
    x, y = find_coorindates(N)
    return manhattan_distance((0, 0), (x, y))


data = 312051
assert carry_distance(1) == 0
assert carry_distance(12) == 3
assert carry_distance(23) == 2
assert carry_distance(1024) == 31
carry_distance(data)

430

The second part requires us to perform a "stress test" to populate a spiral grid with values the sum of all adjacent cells in the grid. As we don't know the upper bound (and I don't know how to initialize an infinite grid that can be referenced arbitarily..) I went with a dictionay to store point references and their values. 

By combining two infinite generators that cycle through the directions we turn and the distances we need to travel, we build an a third infinite generator that contains all the steps we'll take.

In [106]:
from itertools import cycle, count

def spiral_distances():
    """"Yields 1, 1, 2, 2, 3, 3, ...
    
    As the spiral wraps around itself, we increase
    the distance we travel by 1 every two distances.
    
    This is because every 2 distances we're moving in
    the opposite direction, so to we increase the
    distance to ensure we can move past the movement
    we're now opposing.
    """
    for distance in count(1):
        yield distance
        yield distance

            
def directions():
    """Yields R, U, L, D, R, U, L, D, ..."""
    up = (0, -1)
    down = (0, 1)
    left = (-1, 0)
    right = (1, 0)
    return cycle((right, up, left, down))


def spiral_movements():
    for distance, direction in zip(spiral_distances(), directions()):
        for _ in range(distance):
            yield direction


def stress_test(max_val):
    grid = {}
    x, y = 0, 0
    grid[(x, y)] = 1
    for direction in spiral_movements():
        dx, dy = direction
        x += dx
        y += dy
        val = sum(
            grid.get(neighbour, 0)
            for neighbour in neighbours8(x, y)
        )

        grid[(x, y)] = val

        if val > max_val:
            return val
stress_test(data)
    

312453

# [Day 4: High-Entropy Passphrases](https://adventofcode.com/2017/day/4)

W're tasked with checking the validity of a series of passphrases. A passphrase is considered valid if each word within the passphrase is unique.

In [151]:
def is_valid(passphrase):
    words = re.findall('\w+', passphrase)
    unique = set(words)
    return len(unique) == len(words)

def valid_passphrases(data):
    return sum(
        is_valid(passphrase)
        for passphrase in data.split('\n')
    )
    

valid_passphrases(Input(4).read())

466

Part two requires us to ensure that no anagrams are present.

In [150]:
def is_valid(passphrase):
    words = re.findall('\w+', passphrase)
    unique = set(
        cat(sorted(w)) for w in words
    )
    return len(unique) == len(words)

assert not is_valid('abcde xyz ecdab')

valid_passphrases(Input(4).read())

251

# [Day 5: A Maze of Twisty Trampolines, All Alike](https://adventofcode.com/2017/day/5)

In this example we're following a series of jump instructions through an list that modify the jump distance once it's been performed. For each item in the list, `n`, we jump `n` steps away and increment the jump value in at that index by 1.

Originally I didn't have the `increment_by` as a callable, and in doing so the program is noticably slowed (part two went from 7s to 9s), however it makes the second part trivial so I introduced it.

In [153]:
def follow_instructions(data, increment_by=lambda n: 1):
    instructions = list(map(int, re.findall('-?\d+', data)))
    step_count = 0
    index = 0
    while True:
        try:
            jump = instructions[index]
        except IndexError:
            break
        instructions[index] += increment_by(jump)
        index += jump
        step_count += 1
    return step_count

test_input = """
0
3
0
1
-3
"""
assert follow_instructions(test_input) == 5

% time follow_instructions(Input(5).read())

CPU times: user 124 ms, sys: 2.4 ms, total: 126 ms
Wall time: 125 ms


359348

The second part is the same as the first but this time we decrement the jump value if it's `>= 3`.

In [154]:
increment_by = lambda n: -1 if n > 2 else 1
assert follow_instructions(test_input, increment_by) == 10
% time follow_instructions(Input(5).read(), increment_by)

CPU times: user 9.36 s, sys: 20.3 ms, total: 9.38 s
Wall time: 9.39 s


27688760

# [Day 6: Memory Reallocation](https://adventofcode.com/2017/day/6)

We're tasked re-bucketing data, and dedecting any cyclical allocations. Given N buckets, take the maximum, set that bucket to zero and then add one for N to each other bucket in order, wrapping to the start.

i.e.

```
    (0, 2, 7, 0) Initial state, 7 is highest so set to zero and distribute
    (2, 4, 1, 2) 4 is now the highest
    (3, 1, 2, 3) We have two threes, the first takes precedence
    (0, 2, 3, 4) 4 is now the hightest
    (1, 3, 4, 1) 4 again wins
    (2, 4, 1, 2) *LOOP* We've seen this state before, and was detected on the 5th step.
```

As I'm generating this data as we go, I don't think there's any nifty algorithms that we can implement to detect a loop. Instead I just keep track of what's been seen and compare the current state.

As we want to use a set to keep track of what we've seen there's a lot of conversion between lists and tuples, but this is a smaller trade off as the seen list can get quite large, so [O(1) member test compared to O(N)](https://wiki.python.org/moin/TimeComplexity) wins! (Using a list here takes ~20 times longer.)

In [254]:
def redistribute_memory(banks):
    seen = set()
    
    for cycle in count(1):
        if banks in seen:
            break
        seen.add(banks)
        banks = redistribute(banks)
    return cycle - 1


def redistribute(banks):
    banks = list(banks)
    blocks = max(banks)
    max_index = banks.index(blocks)
    
    banks[max_index] = 0
    
    offset = max_index + 1
    addition = blocks // len(banks)
    extra_lim = blocks % len(banks)
    
    for i in range(len(banks)):
        idx = (offset + i) % len(banks)
        banks[idx] += addition
        if i < extra_lim:
            banks[idx] += 1
    return tuple(banks)

In [255]:
assert redistribute((0, 2, 7, 0)) == (2, 4, 1, 2)
assert redistribute_memory((0, 2, 7, 0)) == 5

In [256]:
banks = tuple(
    int(num)
    for num in re.findall('\d+', Input(6).read())
)
redistribute_memory(banks)

14029

For part two we need to know the length of the loop. We can do this by keeping track of the original index of each memory bank and compare it to the index we're on when we reach the final bank.

In [257]:
def redistribute_memory2(banks):
    seen = {}
    
    for cycle in count(1):
        if banks in seen:
            break
        seen[banks] = cycle
        banks = redistribute(banks)
    return cycle - seen[banks] - 1

In [259]:
redistribute_memory2(banks)

2764

# [Day 7: Recursive Circus](http://adventofcode.com/2017/day/7)

Here we meet our first tree. This could probably be solved without such large data structures, but this may come up later so I think it's worth investing time into now.

In [63]:
class WeightedNode(Node):
    def __init__(self, name, weight):
        self.weight = weight
        super().__init__(name)
        
    @property
    def decendant_weight(self):
        return sum(child.weight for child in self.decendants)
    
    @property
    def total_weight(self):
        return self.weight + self.decendant_weight


def parse_input(data):
    chunks = re.findall("(\w+) \((\d+)\)(?: -> ([a-z, ]+))?", data)
    
    nodes = {
        name: WeightedNode(name, int(weight))
        for name, weight, _
        in chunks
    }
    
    for name, _, children in chunks:
        if not children:
            continue
        node = nodes[name]
        for child_name in children.split(', '):
            nodes[child_name].add_parent(node)

    # We can use any node here.
    return node.root

In [59]:
test_input = """
pbga (66)
xhth (57)
ebii (61)
havc (66)
ktlj (57)
fwft (72) -> ktlj, cntj, xhth
qoyq (66)
padx (45) -> pbga, havc, qoyq
tknk (41) -> ugml, padx, fwft
jptl (61)
ugml (68) -> gyxo, ebii, jptl
gyxo (61)
cntj (57)
"""

test_root = parse_input(test_input)
assert test_root.name == 'tknk'

In [61]:
root = parse_input(Input(7).read())
root.name

'hmvwl'

For the second part I needed to find the node lowest in the tree that has a different total weight than it's siblings and calculate what the weight _should_ be. As we know there's only a single node that's the problem, from the root we can identify the node that's incorrect and focus on that subtree until it's no longer incorrect.

In [62]:
def find_total_weights(node):
    while True:
        weights = [
            child.total_weight
            for child in node.children
        ]
        if len(set(weights)) == 1:
            # No problems, all weights are the same.
            # The problematic node was in the previous
            # iteration.
            break
        
        # There are two distinct values, take the second
        # most common to give us the odd one out.
        incorrect_value = Counter(weights).most_common(2)[1][0]
        incorrect_index = weights.index(incorrect_value)
        node = node.children[incorrect_index]

    target_weight = next(node.siblings).total_weight
    diff = target_weight - node.total_weight
    return node.weight + diff


find_total_weights(root)

1853

# [Day 8: I Heard You Like Registers](http://adventofcode.com/2017/day/8)

For this puzzle we have to perform a series of conditional operations based on the values of some "register". All of python's conditional operations call magic methods under the hood. i.e. `A > B` is transformed to `A.__gt__(b)`. These operators are mapped to functions within the operator module.

As we don't know the number of registers that will be referenced, I use a default dict that can grow to whatever's required. The addition of the `__MAX__` flag is for the second part of the question, which wants to know what the largest value seen was.

In [123]:
from operator import gt, lt, ge, le, eq, ne

def perform_instructions(instructions):
    instructions = list(parse_instructions(instructions))
    regs =  defaultdict(int)
    
    for reg, op, amount, test in instructions:
        test_reg, test_op, test_val = test
        
        if not test_op(regs[test_reg], test_val):
            # Test failed, skip operation
            continue
            
        if   op == 'inc': regs[reg] += amount
        elif op == 'dec': regs[reg] -= amount
            
        if regs[reg] > regs['__MAX__']:
            regs['__MAX__'] = regs[reg]

    return regs


def parse_instructions(instructions):
    op_map = {
        '>': gt,
        '<': lt,
        '>=': ge,
        '<=': le,
        '==': eq,
        '!=': ne
    }
    for d in re.findall('(\w+) (\w+) (-?\d+) if (\w+) ([><=!]+) (-?\d+)', instructions):
        reg, op, amount, test_reg, test_op, test = d
        yield (
            reg, op, int(amount), (test_reg, op_map[test_op], int(test))
        )
       
    
max_reg_val = lambda regs: max(
    val
    for key, val in regs.items()
    if key != '__MAX__'
)

In [124]:
test_instructions = """
b inc 5 if a > 1
a inc 1 if b < 5
c dec -10 if a >= 1
c inc -20 if c == 10
"""
regs = perform_instructions(test_instructions)
assert max_reg_val(regs) == 1

In [125]:
regs = perform_instructions(Input(8).read())
max_reg_val(regs)

5143

In [126]:
regs['__MAX__']

6209